In [1]:
import os

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

In [10]:
YEAR_CODES = {
    1951: {
        'num_pop_tot': 'agec_tot1951ttd',
        'num_imm_tot': '',
        'num_jew_tot': 'eth_euroothrjewi1951tt1',
        'num_chn_tot': 'eth_asia_tot1951tt1',  # Asian, not Chinese -- but Chinese are a majority in 1951
    }, 
    1956: {
        'num_pop_tot': 'agec_tot1956ttd',
        'num_imm_tot': '',
        'num_jew_tot': '',
        'num_chn_tot': '',
    }, 
    1961: {
        'num_pop_tot': 'agec_tot1961ttd',
        'num_imm_tot': 'imb__tot1961ttd',
        'num_jew_tot': 'eth_euroothrjewi1961tt1',
        'num_chn_tot': 'eth_asiaeastchin1961tt1',
    }, 
    1966: {
        'num_pop_tot': 'agec_tot1966ttd',
        'num_imm_tot': '',
        'num_jew_tot': '',
        'num_chn_tot': '',
    }, 
    1971: {
        'num_pop_tot': 'agec_tot1971ttd',
        'num_imm_tot': 'imb__tot1971ttd',
        'num_jew_tot': 'eth_euroothrjewi1971tt1',
        'num_chn_tot': 'eth_asiaeastchin1971tt1',
    }, 
    1976: {
        'num_pop_tot': 'agec_tot1976ttd',
        'num_imm_tot': '',
        'num_jew_tot': '',
        'num_chn_tot': '',
    }, 
    1981: {
        'num_pop_tot': 'agec_tot1981ttd',
        'num_imm_tot': 'imag_tot1981ttd',
        'num_jew_tot': '',
        'num_chn_tot': '',
    }, 
    1986: {
        'num_pop_tot': 'agec_tot1986ttd',
        'num_imm_tot': 'imb__tot1986ttd',
        'num_jew_tot': 'eth_euroothrjewi1986tt1',
        'num_chn_tot': 'eth_asiaeastchin1986tt1',
    }, 
    1991: {
        'num_pop_tot': 'agec_tot1991ttd',
        'num_imm_tot': 'imd__tot1991ttd',
        'num_jew_tot': 'eth_euroothrjewi1991tt1',
        'num_chn_tot': 'eth_asiaeastchin1991tt1',
    }, 
    1996: {
        'num_pop_tot': 'agec_tot1996ttd',
        'num_imm_tot': 'imb__tot1996ttd',
        'num_jew_tot': 'ethmeuroothrjewi1996tt1',
        'num_chn_tot': 'ethmasiaeastchin1996tt1',
    }, 
    2001: {
        'num_pop_tot': 'agec_tot2001ttd',
        'num_imm_tot': 'imb__tot2001ttd',
        'num_jew_tot': 'ethmeuroothrjewi2001tt1',
        'num_chn_tot': 'ethmasiaeastchin2001tt1',
    }, 
    2006: {
        'num_pop_tot': 'agec_tot2006ttd',
        'num_imm_tot': 'imb__tot2006ttd',
        'num_jew_tot': 'ethmeuroothrjewi2006tt3',
        'num_chn_tot': 'ethmasiaeastchin2006tt2',
    }, 
    2011: {
        'num_pop_tot': 'agec_tot2011ttd',
        'num_imm_tot': 'imb__tot2011ttd',
        'num_jew_tot': 'ethmeuroothrjewi2011tt3',
        'num_chn_tot': 'ethmasiaeastchin2011tt3',
    }, 
    2016: {
        'num_pop_tot': 'agec_tot2016ttd',
        'num_imm_tot': 'imb__tot2016ttd',
        'num_jew_tot': 'ethmeuroothrjewi2016tt3',
        'num_chn_tot': 'ethmasiaeastchin2016tt3',
    }, 
    2021: {
        'num_pop_tot': 'agec_tot2021ttd',
        'num_imm_tot': 'imb__tot2021ttd',
        'num_jew_tot': 'ethmeuroothrjewi2021tt1',
        'num_chn_tot': 'ethmasiaeastchin2021tt1',
    },
}

In [3]:
for year in tqdm(YEAR_CODES.keys()):
    out_dir = f'../data/immigration/{year}'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    out_dir = f'../data/ethnicity/{year}'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

100%|████████████████████████████████████████| 15/15 [00:00<00:00, 10997.13it/s]


In [132]:
def get_target_prop(year, df_cen, tag_total, tag_part, tag_prop_name):
    df_cur_cen = df_cen[['geosid', 'geopart', tag_total, tag_part]].copy(deep=True)
    df_cur_cen[tag_part] = np.where(df_cur_cen[tag_part] < 0, np.nan, df_cur_cen[tag_part])
    df_cur_cen[tag_prop_name] = df_cur_cen[tag_part] / df_cur_cen[tag_total]
    df_cur_cen[tag_prop_name] = df_cur_cen[tag_prop_name].where((0 <= df_cur_cen[tag_prop_name]) & (df_cur_cen[tag_prop_name] <= 1))

    gdf_cur_geo = gpd.read_file(f"../data/geo/{year}_ct_shp/ct_{year}.zip")

    return pd.merge(gdf_cur_geo, df_cur_cen, on="geosid", how='outer')

In [133]:
for year in tqdm(YEAR_CODES.keys()):
    df_cen = pd.read_csv(f"../data/census/{year}_ct_wide/census_wide_{year}_ct.csv", dtype={'geosid': str})
    code_pop_tot = YEAR_CODES[year]['num_pop_tot']
    
    # Immigration
    code_imm_tot = YEAR_CODES[year]['num_imm_tot']
    if (code_pop_tot and code_imm_tot):
        f_name = f'../data/immigration/{year}/imm_prop_{year}.gpkg'
        get_target_prop(year, df_cen, code_pop_tot, code_imm_tot, 'imm_prop').to_file(f_name, driver='GPKG')

    # Ethnicity - Jewish 
    # code_jew_tot = YEAR_CODES[year]['num_jew_tot']
    # if (code_pop_tot and code_jew_tot):
    #     df_cur_cen = df_cen[['geosid', 'geopart', code_pop_tot, code_jew_tot]].copy(deep=True)
    #     df_cur_cen['jew_prop'] = df_cur_cen[code_jew_tot] / df_cur_cen[code_pop_tot]
    
    #     gdf_cur_geo = gpd.read_file(f"../data/geo/{year}_ct_shp/ct_{year}.zip")
    
    #     pd.merge(gdf_cur_geo, df_cur_cen, on="geosid").to_file(f'../data/ethnicity/{year}/jew_prop_{year}.gpkg', driver='GPKG')

    # Ethnicity - Chinese
    # code_chn_tot = YEAR_CODES[year]['num_chn_tot']
    # if (code_pop_tot and code_chn_tot):
    #     df_cur_cen = df_cen[['geosid', 'geopart', code_pop_tot, code_chn_tot]].copy(deep=True)
    #     df_cur_cen['chn_prop'] = df_cur_cen[code_chn_tot] / df_cur_cen[code_pop_tot]
    
    #     gdf_cur_geo = gpd.read_file(f"../data/geo/{year}_ct_shp/ct_{year}.zip")
    
    #     pd.merge(gdf_cur_geo, df_cur_cen, on="geosid").to_file(f'../data/ethnicity/{year}/chn_prop_{year}.gpkg', driver='GPKG')

100%|███████████████████████████████████████████| 15/15 [00:25<00:00,  1.72s/it]
